# Example Network

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Generate synthetic dataset
X, y = make_moons(n_samples=1000, noise=0.2, random_state=0)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define simple NN
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    preds = (model(X_test) > 0.5).float()
    accuracy = (preds.eq(y_test).sum() / len(y_test)).item()
print(f'Test Accuracy: {accuracy:.4f}')


Test Accuracy: 0.8433


# Define Attack Setup

In [4]:
from framework.attack import injector
from framework.criterion import binary_classifcation_accuracy
import pandas as pd

inj = injector(model, X_test, y_test, binary_classifcation_accuracy)

results = inj.run_seu(0)

display(pd.DataFrame(results).head())

results = inj.run_stochastic_seu(0,0.3)

display(pd.DataFrame(results).head())

Testing a forward pass on cuda...
Basline Criterion Score: 0.8433333333333334


,tensor_location,criterion_score,layer_name,value_before,value_after
0,"(0, 0)",0.843333,net.0.weight,0.608715,-0.608715
1,"(0, 1)",0.770000,net.0.weight,-1.257187,1.257187
2,"(1, 0)",0.833333,net.0.weight,1.366734,-1.366734
3,"(1, 1)",0.843333,net.0.weight,-0.347827,0.347827
4,"(2, 0)",0.853333,net.0.weight,-0.570746,0.570746


,tensor_location,criterion_score,layer_name,value_before,value_after
0,"(0, 1)",0.770000,net.0.weight,-1.257187,1.257187
1,"(1, 0)",0.833333,net.0.weight,1.366734,-1.366734
2,"(1, 1)",0.843333,net.0.weight,-0.347827,0.347827
3,"(3, 1)",0.843333,net.0.weight,0.223027,-0.223027
4,"(0,)",0.740000,net.0.bias,1.601659,-1.601659
